In [11]:
import os
import imageio
import numpy as np

In [13]:
path = '../ColabNotebooks/dataset'

In [16]:
def extractFrames(file_name, frame_step):
  video = imageio.get_reader(file_name, 'ffmpeg')
  return np.array([frame for i, frame in enumerate(video) if i % frame_step == 0])

In [17]:
def preProcessing(frames):
  video_r_channel = np.zeros(frames.shape[0:3])
  video_g_channel = np.zeros(frames.shape[0:3])
  video_b_channel = np.zeros(frames.shape[0:3])

  for i, frame in enumerate(frames):
    for j, row in enumerate(frame):
      for k, pixel in enumerate(row):
        video_r_channel[i][j][k] = pixel[0]
        video_g_channel[i][j][k] = pixel[1]
        video_b_channel[i][j][k] = pixel[2]
        
  return video_r_channel, video_g_channel, video_b_channel

In [18]:
# Returns correlation value
def superimpose(frame: np.ndarray, kernel: np.ndarray):
  n = kernel.shape[0]
  mid = n // 2

  superimposed_matrix = frame * kernel
  return sum(superimposed_matrix.flatten()) - superimposed_matrix[mid][mid]

In [19]:
def isValid(index, length, skip):
  return index >= skip and index < length - skip

In [20]:
# Channel is a 3D matrix: (number of frames, height, width)
# each element represents the pixel value of the given channel

def convolve(channel, kernel):
  n = len(kernel) # Kernel is of dimension n x n
  skip = n // 2 # No. of rows and cols to be skipped

  num_frames, height, width = channel.shape

  # res_frames = np.zeros((len(frames), height - 2 * skip, width - 2 * skip))
  res_channel = np.zeros(channel.shape)

  for i, frame in enumerate(channel):
    print(f'Frame: {i}')

    for j, row in enumerate(frame):
      if not isValid(j, height, skip):
        continue

      for k, pixel in enumerate(row):
        if not isValid(k, width, skip):
          continue

        subframe = frame[j - skip: j + skip + 1, k - skip: k + skip + 1]

        res_channel[i][j][k] = superimpose(subframe, kernel)

  return res_channel[:, skip:-skip, skip:-skip]

In [21]:
def saveAsVideo(frames, file_name, fps):
  frames = frames.astype('uint8')
  writer = imageio.get_writer(file_name, fps=fps, macro_block_size=None)
  for frame in frames:
    writer.append_data(frame)

In [23]:
#driver code
files = os.listdir(path)

frames = np.array(extractFrames(os.path.join(path, files[5]), 30))
video_r_channel, video_g_channel, video_b_channel = preProcessing(frames)

In [ ]:
saveAsVideo(video_r_channel, 'video_r.mp4', 5)
saveAsVideo(video_g_channel, 'video_g.mp4', 5)
saveAsVideo(video_b_channel, 'video_b.mp4', 5)

In [24]:
tmpframe = np.array([[[2,2,2,4,5,6], [2,2,2,4,5,6], [2,2,2,4,5,6], [2,2,2,4,5,6], [1,2,3,4,5,6]]])
print(tmpframe.shape)

# tmpkernel = np.ones((3,3))
tmpkernel = np.full((3,3), 1/9)

tmpa = convolve(tmpframe, tmpkernel)
print(tmpa)

(1, 5, 6)
Frame: 0
[[[1.77777778 2.44444444 3.22222222 4.44444444]
  [1.77777778 2.44444444 3.22222222 4.44444444]
  [1.77777778 2.55555556 3.33333333 4.44444444]]]


In [25]:
kernel = np.full((3,3), 1/9)
actual = video_r_channel[0][1:-1, 1:-1]
actual

array([[163., 163., 163., ..., 158., 158., 158.],
       [163., 163., 163., ..., 158., 158., 158.],
       [163., 163., 163., ..., 159., 159., 159.],
       ...,
       [ 52.,  51.,  51., ..., 253., 253., 253.],
       [ 52.,  52.,  52., ..., 253., 253., 253.],
       [ 52.,  52.,  52., ..., 253., 253., 253.]])

In [26]:
predicted = convolve(np.array([video_r_channel[0]]), kernel).astype('uint8')
predicted

Frame: 0


array([[[144, 144, 144, ..., 140, 140, 140],
        [144, 144, 144, ..., 140, 140, 140],
        [144, 144, 144, ..., 140, 140, 140],
        ...,
        [ 45,  45,  45, ..., 224, 224, 224],
        [ 46,  46,  45, ..., 224, 224, 224],
        [ 47,  46,  46, ..., 224, 224, 224]]], dtype=uint8)

In [ ]:
imageio.imwrite('predicted_image_r.jpg', predicted[0])

In [27]:
diff = abs(actual - predicted[0]).astype('uint8')
diff

array([[19, 19, 19, ..., 18, 18, 18],
       [19, 19, 19, ..., 18, 18, 18],
       [19, 19, 19, ..., 19, 19, 19],
       ...,
       [ 7,  6,  6, ..., 29, 29, 29],
       [ 6,  6,  7, ..., 29, 29, 29],
       [ 5,  6,  6, ..., 29, 29, 29]], dtype=uint8)

In [28]:
(1 / len(diff.flatten())) * sum(diff.flatten())

13.366895691824933

In [ ]:
imageio.imwrite('error_image_r.jpg', diff)

# Below Code is Work-in-Progress

In [29]:
from sklearn.mixture import GaussianMixture

In [61]:
gm = GaussianMixture(n_components=1, random_state=45)
tmpX = np.array([[1], [2], [3], [4], [5]])
tmpModel = gm.fit(tmpX)
tmpMeans = tmpModel.means_.flatten()[0]
tmpStddev = (tmpModel.covariances_ ** (1/2)).flatten()[0]

print(tmpMeans, tmpStddev)

2.999999999999999 1.414213915926441


In [71]:
import scipy.stats
tmpObj = scipy.stats.norm(loc=tmpMeans, scale=tmpStddev)
tmpObj.pdf(2.9999)

0.2820947205449703

In [75]:
1 / (tmpStddev * (((2 * np.pi) ** 0.5)))
# You're muted

0.28209472125020674

In [ ]:
# i will call you again
